# 1 - Simple Expanding Sparse Associative Memory (ESAM) 
 
This notebook demonstrates how to instantiate and run a simple ESAM network.

It's a good place to start if you would like to experiment with the network and its parameters.

## Dependencies

In [1]:
import sys
sys.path.append('../src')


## Define the problem space

The problem space describes the data that will be generated. This dictionary corresponds to **Table 2** in the paper.

* `f`: The length of each signal. This defines the nature of the data that will be automatically generated for the test and also the shape of the network.
* `m`: The number of memories. The network will be generated according to these memories.
* `s_m`: Memory sparsity. The sparsity of each memory defined as a proportion of features that are active for the memory.
* `s_n`: The noise sparsity defined as a proportion of features to be flipped (one to zero, or zero to one) to convert a memory to a recall signal.

In [2]:
f = 900
problem_definition = {'f': f,
                      'm': 200,
                      's_m': 0.2,
                      's_n': 0.05}

## Define the network

The network describes the characteristics of a network that has previously undergone learning. From this description and the memory signals, a fully trained network will be generated. 

The network description comprises the following parameters. These parameters are described in **Tables 3, 4, and 5** in the paper.

* `f`:  The length of each signal. This defines the nature of the data that will be automatically generated for the test and also the shape of the network.
* `h`:  The number of hidden neurons per memory stored. This assumes perfect linear neurogenesis where exactly `h` hidden neurons have matured per memory.  
* `f_h_sparsity`: Sparsity of the feature to hidden neuron synaptic connections. See also `f_h_conn_type`.
* `h_f_sparsity_e`: Sparsity of the hidden to feature neuron excitatory synaptic connections. See also `h_f_conn_type`.
* `e`: The number of recall epochs (iterations).
* `h_thresh`: The threshold assigned to every hidden neuron. Each hidden neuron pre-synaptic connection carries a weight of `1` and the bias associated with each hidden neuron is `1`. During the sub-pattern detection step, a hidden neuron will fire when the number of active pre-synaptic connections is equal to or exceeds the threshold.
* `f_h_conn_type`: `FixedNumber` ensures a fixed number of pre-synaptic connections `f * f_h_sparsity * s_m` for each hidden neuron. `FixedProbability` means that the probability of a connection from a feature neuron to a hidden neuron is `f_h_sparsity * s_m`. The latter results in the number of pre-synaptic connections varying according to a binomial distribution across the hidden neuron population.

In [3]:
network_definition = {'f': f,
                      'h': 2,
                      'f_h_sparsity': 0.1,
                      'h_f_sparsity_e': 1,
                      'h_f_sparsity_i': 1,
                      'e': 3,
                      'h_thresh': 12,
                      'f_h_conn_type': 'FixedProbability',
                      'h_f_conn_type': 'FixedProbability'}

## Generate the data 

The data is generated based on the problem space definition provided previously. 

In [4]:
from data_generation.data_generator import generate_simulation_data

tests_per_memory = 1
num_simulations = problem_definition['m'] * tests_per_memory
memories, tests, ground_truth_labels = generate_simulation_data(problem_space=problem_definition,
                                                                num_simulations=num_simulations)

======= Generating Simulation Data =======
      Number of features (feature neurons): 900
      Number of memories:                   200
      Number of simulations:                200
      Signal length:                        180
      Bits to flip for noise:               45


## Generate the trained network

Generate the ESAM network based on the characterisation provided previously.

The network is created based on the network definition.

In [5]:
from binary_esam import BinaryESAM
esam = BinaryESAM(network_params=network_definition)


=========== ESAM Recall Network =============
    Num features:                   900
    F to H connection sparsity:     0.1
      Static F to H conn type?      FixedProbability
    H to F exitatory sparsity:      1
    H to F inhibitory sparsity:     1
      Static H to F conn type?      FixedProbability
    Max epochs per recall:          3
    Hidden per memory:              2
    Hidden threshold:               12



Train the network with the set of perfect memories. These should form attractor points.

This is a simulated 'learning' step because the network is manipulated as if a learning algorithm had occurred. Refer to pproach described in the paper for further details.

In [6]:
esam.learn(memories)

>> Learning 200 memories
200 memories learnt in 0.01s. Approx time per memory: 0.05ms


True

## Run the tests

Pass each of the noisy memory variants into the ESAM. Each should resolve to its original memory. The label (which identifies the ground truth memory) is also passed for each test. 

The recall function returns a `SimulationReporter` that encapsulates all the results for analysis.

In [7]:
reporter = esam.recall(tests, ground_truth_labels)

print('\nProportion of noisy signals that correctly resolved to their memory: {}'.format(reporter.get_proportion_correct_recall()))


>> Recalling 200 tests.
Recall complete. Average time per test (ms) was:  4.25

Proportion of noisy signals that correctly resolved to their memory: 0.885


## Test Diagnostics

Print diagnostics for each test.

### Averages per Epoch

Display the averages across all the tests for each epoch. This comprises:

* `av_signal_length`: Average number of active features (1s) in the signal for the epoch.

Runtime statistics for the sub-pattern recognition step:

* `f_h_spikes`: Average number of hidden neuron pre-synaptic spikes for this epoch - that is the number of spikes during the sub-pattern recognition step.
* `f_h_spikes_so_far`: Average total number of feature neuron pre-synaptic spikes so far.
* `num_spiking_h`: Average number of hidden neurons that spiked for this epoch. Epoch 0 represents the input message, so there is no spiking on this epoch.
* `prop_h_gt`: Avereage proportion of the ground truth hidden population that spiked correctly for this epoch. This is the total number of hidden neurons that spiked correctly (were associated with the ground truth) divided by the number of hidden neurons associated with each ground truth `h`.
* `prop_h_not_gt`: Average proportion of the hidden population that are not in the gound truth set that spiked (incorrectly). This is the total number of hidden neurons that spiked correctly (were not associated with the ground truth) divided by the number of hidden neurons not associated with each ground truth `h * (num_mems - 1)`.

Runtime statistics for the modulation step:

* `f_h_spikes`: Average number of feature neuron pre-synaptic spikes for this epoch - that is the number of spikes during the modulation step.
* `f_h_spikes_so_far`: Average total number of feature neuron pre-synaptic spikes so far.
* The average number of feature neurons spiking is the same as the `av_signal_length` for the next epoch.




In [8]:
from IPython.display import display, HTML

averages_per_epoch = reporter.get_test_data_averages(problem_space=problem_definition)
averages_per_epoch.index.name = 'epoch num'
data_to_display = averages_per_epoch[['av_signal_length', 'f_h_spikes', 'f_h_spikes_so_far', 'num_spiking_h', 'prop_h_gt', 'prop_h_not_gt', 'f_h_spikes', 'f_h_spikes_so_far']]
display(HTML(data_to_display.to_html()))

,av_signal_length,f_h_spikes,f_h_spikes_so_far,num_spiking_h,prop_h_gt,prop_h_not_gt,f_h_spikes,f_h_spikes_so_far
epoch num,,,,,,,,
0,207,0,0,0.0,0.000,0.000000,0,0
1,165,1702,1702,2.305,0.935,0.001093,1702,1702
2,166,1359,3061,1.86,0.860,0.000352,1359,3061
3,167,1367,4428,1.88,0.865,0.000377,1367,4428


### All Simulations - Detailed Test Diagnostics

Display more detailed results for every simulation for each test

* `signal_length`: Number of active features (1s) in the signal for the epoch.
* `gt_memory_index`: Ground truth index for the simulation.
* `gt_mem_diffs`: Sum of the XOR of current epoch signal with the ground truth. This will be 0 when the ground truth is correctly retrieved.
* `f_h_spikes`: Number of hidden neuron pre-synaptic spikes.
* `h_gt`: Number of ground truth neurons that spiked correctly.
* `h_not_gt`: Number of none ground truth neurons that spiked (incorrectly).
* `prop_h_gt`: Proportion of the ground truth hidden population that spiked correctly.
* `prop_h_not_gt`: Proportion of the hidden population that are not in the gound truth set that spiked (incorrectly).

In [9]:
all_data = reporter.get_test_data_all_simulations()
for simulation_num in range(num_simulations):
    print('\nSimulation number: {}'.format(simulation_num))
    sim_data = all_data.loc[(all_data['sim_num'] == simulation_num)].copy()
    sim_data.drop('sim_num', axis=1, inplace=True)
    sim_data.reset_index(drop=True, inplace=True)
    sim_data.set_index('epoch')
    display(HTML(sim_data.to_html()))


Simulation number: 0


,epoch,correct,incorrect,signal_length,gt_mem_index,gt_mem_diffs,f_h_spikes,h_gt,h_not_gt,prop_h_gt,prop_h_not_gt
0,0,0,True,213,0,45,0,0,0,0.0,0.0
1,1,1,False,180,0,0,1714,2,0,1.0,0.0
2,2,1,False,180,0,0,1466,2,0,1.0,0.0
3,3,1,False,180,0,0,1466,2,0,1.0,0.0



Simulation number: 1


,epoch,correct,incorrect,signal_length,gt_mem_index,gt_mem_diffs,f_h_spikes,h_gt,h_not_gt,prop_h_gt,prop_h_not_gt
0,0,0,True,215,1,45,0,0,0,0.0,0.0
1,1,1,False,180,1,0,1740,2,0,1.0,0.0
2,2,1,False,180,1,0,1463,2,0,1.0,0.0
3,3,1,False,180,1,0,1463,2,0,1.0,0.0



Simulation number: 2


...

,epoch,correct,incorrect,signal_length,gt_mem_index,gt_mem_diffs,f_h_spikes,h_gt,h_not_gt,prop_h_gt,prop_h_not_gt
0,0,0,True,201,197,45,0,0,0,0.0,0.0
1,1,1,False,180,197,0,1603,2,0,1.0,0.0
2,2,1,False,180,197,0,1450,2,0,1.0,0.0
3,3,1,False,180,197,0,1450,2,0,1.0,0.0



Simulation number: 198


,epoch,correct,incorrect,signal_length,gt_mem_index,gt_mem_diffs,f_h_spikes,h_gt,h_not_gt,prop_h_gt,prop_h_not_gt
0,0,0,True,209,198,45,0,0,0,0.0,0.0
1,1,1,False,180,198,0,1663,2,0,1.0,0.0
2,2,1,False,180,198,0,1465,2,0,1.0,0.0
3,3,1,False,180,198,0,1465,2,0,1.0,0.0



Simulation number: 199


,epoch,correct,incorrect,signal_length,gt_mem_index,gt_mem_diffs,f_h_spikes,h_gt,h_not_gt,prop_h_gt,prop_h_not_gt
0,0,0,True,211,199,45,0,0,0,0.0,0.000000
1,1,1,False,180,199,0,1707,2,1,1.0,0.002513
2,2,1,False,180,199,0,1447,2,0,1.0,0.000000
3,3,1,False,180,199,0,1447,2,0,1.0,0.000000
